In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, f1_score, precision_score, recall_score, accuracy_score
import seaborn as sns
# warnings.filterwarnings('ignore')


In [2]:
df = pd.read_csv("data/sqliv2.csv", encoding='utf-16')
print("Data Shape:", df.shape)
df.Label.unique()
df[df['Label']==0]

Data Shape: (33761, 2)


,Sentence,Label
951,NaN,0
952,When I &apos;m finished patient write-ups nig...,0
953,"Thanks mooring basin front terminal , sunken ...",0
954,Geisingen Kirchen-Hausen first documented tog...,0
955,"However , visitors marvel two ornate church t...",0
...,...,...
33756,syrett,0
33757,arrechea bellveh,0
33758,1664,0
33759,almaluez,0


In [4]:
X = df['Sentence']
y = df['Label']
print(X.shape, y.shape)
len(X)
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
nltk.download('stopwords')
vectorizer = CountVectorizer(min_df = 2, max_df = 0.8, stop_words = stopwords.words('english'))
X = vectorizer.fit_transform(X.values.astype('U')).toarray()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(33761,) (33761,)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\janha\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


(27008, 10321)
(27008,)
(6753, 10321)
(6753,)


In [10]:
num_features = X_train.shape[1]
num_features

10321

In [11]:
import tensorflow as tf
from tensorflow.keras.utils import plot_model
from tensorflow.keras import models, layers
# import warnings

from sklearn.metrics import confusion_matrix, classification_report, f1_score, precision_score, recall_score, accuracy_score
from sklearn.metrics import precision_recall_curve,precision_recall_fscore_support


model = models.Sequential()
model.add(layers.Conv1D(32, 1, activation = 'relu', input_shape = (1,num_features)))
model.add(layers.Conv1D(32, 1, activation = 'relu'))
model.add(layers.Flatten())
model.add(layers.Dense(1, activation = 'sigmoid'))
model.summary()
model.compile(optimizer = 'adam', loss = tf.keras.losses.BinaryCrossentropy(), metrics = ['accuracy'])

X_train1 = X_train.reshape(-1, 1, num_features)
X_test1 = X_test.reshape(-1, 1, num_features)

history = model.fit(X_train1, y_train, epochs = 10, validation_data = (X_test1, y_test))
y_pred = model.predict(X_test1).flatten()

y_pred = np.round(y_pred)

model.save("saveModel/")


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 1, 32)             330304    
                                                                 
 conv1d_1 (Conv1D)           (None, 1, 32)             1056      
                                                                 
 flatten (Flatten)           (None, 32)                0         
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 331,393
Trainable params: 331,393
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
844/844 [==============================] - 16s 15ms/step - loss: 0.1778 - accuracy: 0.9496 - val_loss: 0.1050 - val_accuracy: 0.9662
Epoch 2/10
844/844 [==================

INFO:tensorflow:Assets written to: saveModel/assets


INFO:tensorflow:Assets written to: saveModel/assets


In [14]:
from tensorflow.keras.models import model_from_json
model_json = model.to_json()
with open("cnn_model_big.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("cnn_model_big.h5")
print("Saved model to disk")


Saved model to disk


In [12]:
print(f"Accuracy of CNN on test set : {accuracy_score(y_pred, y_test)}")
print(f"F1 Score of CNN on test set : {f1_score(y_pred, y_test)}")
print(f"recall of CNN on test set : {recall_score(y_pred, y_test)}")
print(f"precision of CNN on test set : {precision_score(y_pred, y_test)}")


Accuracy of CNN on test set : 0.9647564045609359
F1 Score of CNN on test set : 0.9461538461538462
recall of CNN on test set : 0.9863207547169811
precision of CNN on test set : 0.9091304347826087


In [15]:
json_file = open('cnn_model_big.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
cnn_model = model_from_json(loaded_model_json)
# load weights into new model
cnn_model.load_weights("cnn_model_big.h5")
print("Loaded model from disk")

Loaded model from disk
